In [1]:
import pickle
import torch
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
model = pickle.load(open('writer_varification_model.sav','rb'))

In [3]:
model

(Sequential(
   (0): Conv2d(1, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   (1): ReLU()
   (2): Conv2d(5, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (5): ReLU()
   (6): Conv2d(15, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (8): Flatten(start_dim=1, end_dim=-1)
   (9): Linear(in_features=30000, out_features=5000, bias=True)
   (10): ReLU()
   (11): Linear(in_features=5000, out_features=1000, bias=True)
 ),
 KNeighborsClassifier(n_neighbors=435))

In [7]:
test_set = pd.read_csv('test.csv')
test_set.head()

,img1_name,img2_name
0,3861e092.jpg,b642bddc.jpg
1,776e0055.jpg,2d4e7d20.jpg
2,4bb7f909.jpg,af1b0fbf.jpg
3,0abee1d8.jpg,1c2641c9.jpg
4,1e18f51e.jpg,9fcf5c47.jpg


In [8]:
output = []
root = 'semi_test/'
l = len(test_set)
model_1 = model[0]
model_2 = model[1]
for i in range(l):
  img_1 = test_set.iloc[i,0]
  img_2 = test_set.iloc[i,1]
  id = img_1+'_'+img_2
  img_1 = Image.open(root+img_1).resize((500,50))
  img_2 = Image.open(root+img_2).resize((500,50))
  X1 = np.array([[np.array(img_1)]])
  X2 = np.array([[np.array(img_2)]])
  with torch.inference_mode(True):
    fea1 = model_1(torch.tensor(X1).float()).cpu().numpy()
    fea2 = model_1(torch.tensor(X2).float()).cpu().numpy()
  diff = np.abs(fea1-fea2)
  proba = model_2.predict_proba(diff)
  output.append((id,proba))
  print(f'\r{100*i/l:.4f}',end='')

99.9906

In [13]:
id = []
proba = []
for i,p in output:
  id.append(i)
  proba.append(p[0][1])

In [15]:
submision = pd.DataFrame([])

In [17]:
submision['id'] = id
submision['proba'] = proba
submision.head()

,id,proba
0,3861e092.jpg_b642bddc.jpg,0.947126
1,776e0055.jpg_2d4e7d20.jpg,0.866667
2,4bb7f909.jpg_af1b0fbf.jpg,0.179310
3,0abee1d8.jpg_1c2641c9.jpg,0.836782
4,1e18f51e.jpg_9fcf5c47.jpg,0.000000


In [20]:
submision.to_csv('Neural Survivor.csv',index = False)